In [1]:
!pip install transformers datasets evaluate accelerate
!pip install torch torchvision torchaudio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np

In [3]:
dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# Select a smaller subset of the training and test data
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Load the accuracy metric from the 'evaluate' library
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """
    Function to compute metrics for evaluation.

    Args:
        eval_pred (tuple): A tuple containing logits and labels.
            - logits (numpy.ndarray): The raw prediction scores from the model.
            - labels (numpy.ndarray): The true labels for the evaluation.

    Returns:
        dict: A dictionary containing the computed accuracy metric.
    """
    logits, labels = eval_pred  # Unpack the tuple into logits and labels

    # Get the predicted class by taking the argmax across the last axis
    predictions = np.argmax(logits, axis=-1)

    # Compute the accuracy metric using the predicted and true labels
    return metric.compute(predictions=predictions, references=labels)


In [8]:
# Define training arguments for the model
training_args = TrainingArguments(
    output_dir="./results",  # Directory where model predictions and checkpoints will be saved
    num_train_epochs=3,  # Number of epochs to train the model
    per_device_train_batch_size=16,  # Batch size for training per device (e.g., GPU or CPU)
    per_device_eval_batch_size=64,  # Batch size for evaluation per device
    warmup_steps=500,  # Number of steps to perform learning rate warm-up
    weight_decay=0.01,  # Weight decay (L2 regularization) to apply during
)

In [9]:
# Create a Trainer instance with the specified configuration
trainer = Trainer(
    model=model,  # The model instance to be trained and evaluated
    args=training_args,  # Training arguments specifying the training configuration
    train_dataset=small_train_dataset,  # The dataset used for training
    eval_dataset=small_eval_dataset,  # The dataset used for evaluation
    compute_metrics=compute_metrics,  # Function to compute metrics during evaluation
)


In [10]:
trainer.train()

Step,Training Loss
500,0.402100
1000,0.230000
1500,0.135800


TrainOutput(global_step=1875, training_loss=0.22129561258951824, metrics={'train_runtime': 1474.7219, 'train_samples_per_second': 20.343, 'train_steps_per_second': 1.271, 'total_flos': 3974021959680000.0, 'train_loss': 0.22129561258951824, 'epoch': 3.0})

In [11]:
trainer.evaluate()

{'eval_loss': 0.3190137445926666,
 'eval_accuracy': 0.917,
 'eval_runtime': 17.42,
 'eval_samples_per_second': 57.405,
 'eval_steps_per_second': 0.918,
 'epoch': 3.0}

In [14]:
# save the model
trainer.save_model("./fine_tuned_model")

In [15]:

# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# List of test examples
test_examples = [
    "This movie was fantastic! I really enjoyed every moment of it.",
    "This movie is average.",
    "First half of the movie is good but second half is like wow.",
    "it is one of the worst movie i have ever seen ",
    "it is the best novie of all the time"
]

# Tokenize the input with padding and truncation
inputs = tokenizer(
    test_examples,
    return_tensors="pt",    # Return as PyTorch tensors
    padding=True,           # Pad the sequences to the model's maximum length
    truncation=True         # Truncate sequences longer than the model's max length
)

# Move the inputs to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Make the model prediction
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted class for each input in the batch
predicted_classes = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Print the results
for i, predicted_class in enumerate(predicted_classes):
    sentiment = 'Positive' if predicted_class == 1 else 'Negative'
    print(f"Statement: {test_examples[i]}")
    print(f"Predicted sentiment: {sentiment}\n")


Statement: This movie was fantastic! I really enjoyed every moment of it.
Predicted sentiment: Positive

Statement: This movie is average.
Predicted sentiment: Negative

Statement: First half of the movie is good but second half is like wow.
Predicted sentiment: Positive

Statement: it is one of the worst movie i have ever seen 
Predicted sentiment: Negative

Statement: it is the best novie of all the time
Predicted sentiment: Positive

